In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
x_train_path = "data/X_train_Hi5.csv"
X_train = pd.read_csv(x_train_path)

In [ ]:
X_train.head(15)

In [ ]:
X_train.info()


In [ ]:
X_train[["piezo_groundwater_level_category"]]

In [ ]:
X_train.sort_values(["piezo_groundwater_level_category"])


In [ ]:
X_train=X_train.rename({"piezo_groundwater_level_category":"target"},axis=1)
X_train.head()

In [ ]:
X_train.sort_values(["target"])

In [ ]:
X_train[['piezo_station_department_code','target']].sort_values(["target"])



In [ ]:
humid_40=X_train.loc[X_train["target"]=="Low","meteo_humidity_duration_below_40%"]
humid_40.dropna()
humid_40.plot()

In [ ]:
humid_80=X_train.loc[X_train["target"]=="High","meteo_humidity_duration_above_80%"]
humid_80.plot()

In [ ]:
howwet=X_train.loc[X_train["target"]=="Very Low","meteo_wetting_duration"]
howwet.dropna()


In [ ]:
level_rain=X_train.loc[X_train["target"]=="Very Low","meteo_rain_height"]
level_rain.plot()

In [ ]:

target_choice = ["Very Low", "Low", "Average", "High", "Very High"]

fig, axes = plt.subplots(5, 1, figsize=(20, 20))  

for i in range(5):  
    usage_type = X_train.loc[X_train["target"] == target_choice[i], "prelev_usage_label_0"]
    usage_type = usage_type.dropna()  
    

    percentages = usage_type.value_counts(normalize=True)
    
    percentages.plot.pie(ax=axes[i], autopct='%1.1f%%',startangle=90, title=f"Usage of the water for stations\nwith groundwater level {target_choice[i]}")
 
plt.tight_layout()
plt.show()



In [ ]:
df_elab= X_train[["hydro_hydro_quantity_elab","hydro_observation_result_elab","hydro_observation_date_elab","hydro_station_code"]]
df_elab.sort_values(["hydro_station_code"])

In [ ]:
#Convertir les dates en nombres de jours
def date_counter(date):
    y_m_d=date.split("-")
    y_m_d=[int(dates) for dates in y_m_d]
    jours=(y_m_d[0]-2020)*365+(y_m_d[1]-1)*30+y_m_d[2]
    return jours

date_counter("2023-01-01")





In [ ]:
df_elab["days"]=df_elab["hydro_observation_date_elab"].apply(date_counter)
df_elab.sort_values(["hydro_station_code","days"])


In [ ]:
df_elab_stations=df_elab.loc[df_elab["hydro_station_code"]=="A022020001" ,["hydro_hydro_quantity_elab","hydro_observation_result_elab","days"]] #only the "elab" variables
df_elab_stations

df_elab_station1=df_elab_stations.loc[df_elab["hydro_hydro_quantity_elab"]=="QmJ"] #only the daily measurements


df_elab_station1.plot(y="hydro_observation_result_elab",x="days",title="Évolution de hydro_quantity_elab sur une station aléatoire")

In [ ]:
#Usage of extracted water as a function of population category

#New variable : size town category
df_extraction= X_train[["prelev_volume_0","prelev_volume_1","prelev_volume_2","prelev_other_volume_sum","prelev_usage_label_0","prelev_usage_label_1","prelev_usage_label_2","insee_pop_commune"]]
df_extraction.dropna()



In [ ]:

def town_category (pop): 
    if pop<=499:
        return "Very small"
    elif pop>=500 and pop<=1999:
        return "Small"
    elif pop>=2000 and pop<=9999:
        return "Average"
    elif pop>=10000 and pop<=49999:
        return "Large"
    elif pop>=50000:
        return "Very Large"
    else:
        return None

df_extraction["pop_category"]=df_extraction["insee_pop_commune"].apply(town_category)
df_extraction['prelev_usage_label_0'] = df_extraction['prelev_usage_label_0'].replace('INDUSTRIE et ACTIVITES ECONOMIQUES (hors irrigation, hors énergie)', 'INDUSTRIE')

effectives = df_extraction["pop_category"].value_counts()
effectives

effectives_counts = df_extraction.groupby(['pop_category', 'prelev_usage_label_0']).size().reset_index(name='effectives')
effectives_counts



In [ ]:

pop_categories = effectives_counts['pop_category'].unique()

# Create subplots
fig, axes = plt.subplots(1, len(pop_categories), figsize=(20, 5), sharey=True)

# Loop through each category and plot the histogram
for i, category in enumerate(pop_categories):
    subset = effectives_counts[effectives_counts['pop_category'] == category]
    axes[i].bar(subset['prelev_usage_label_0'], subset['effectives'], color='blue', alpha=0.7)
    
    axes[i].set_title(f"Usage of extracted water in {category} towns")
    axes[i].set_xlabel("prelev_usage_label_0")
    axes[i].set_ylabel("Effectives")
    axes[i].set_xticks(subset['prelev_usage_label_0'])
    axes[i].set_xticklabels(subset['prelev_usage_label_0'], rotation=45)

plt.tight_layout()
plt.show()

**Computing the indicator of water shortage risk**

In [ ]:
##Relevant variables
#1 Population
df_indicator=X_train[[]]